In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Embedding, Reshape, Conv1D, MaxPool1D, Dropout
from tensorflow.keras.preprocessing.text import one_hot,Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [3]:
data = pd.read_csv('data.txt')

In [4]:
data.head()

,1
0,ای رستخیز ناگهان، وی رحمت بی منتها\tای آتشی اف...
1,امروز خندان آمدی، مفتاح زندان آمدی\tبر مستمندا...
2,خورشید را حاجب تویی، امید را واجب تویی\tمطلب ت...
3,در سینه ها برخاسته، اندیشه را آراسته\tهم خویش ...
4,ای روح بخش بی بَدَل، وی لذتِ علم و عمل\tباقی ب...


In [5]:
with open('stopwords.txt') as stopwords_file:
  stopwords = stopwords_file.readlines()
stopwords = [line.replace('\n', '') for line in stopwords] 

In [6]:
stopwords[0:10]

['!', '"', '#', '(', ')', '*', ',', '-', '.', '/']

In [7]:
len(stopwords)

1316

In [8]:
text = [line.replace('\t', ' ') for line in data.values.flatten()]
text = [line.replace('-', ' ') for line in text]
text = [re.sub("\d+", "", t) for t in text]

In [9]:
text[0:15]

['ای رستخیز ناگهان، وی رحمت بی منتها ای آتشی افروخته، در بیشه ی اندیشه ها',
 'امروز خندان آمدی، مفتاح زندان آمدی بر مستمندان آمدی، چون بخشش و فضل خدا',
 'خورشید را حاجب تویی، امید را واجب تویی مطلب تویی طالب تویی، هم منتها، هم مبتدا',
 'در سینه ها برخاسته، اندیشه را آراسته هم خویش حاجت خواسته، هم خویشتن کرده روا',
 'ای روح بخش بی بَدَل، وی لذتِ علم و عمل باقی بهانه ست و دغل، کاین علت آمد، وآن دوا',
 'ما زان دغل کژ بین شده، با بی گنه در کین شده گه مست حورالعین شده، گه مست نان و شوربا',
 'این سُکر بین، هل عقل را، وین ُنقل بین، هل َنقل را کز بهر نان و بقل را، چندین نشاید ماجرا',
 'تدبیر صد رنگ افکنی، بر روم و بر زنگ افکنی واندر میان جنگ افکنی، فی اصطناع لا یری',
 'میمال پنهان گوش جان، مینه بهانه بر کسان جان رب خلصنی زنان، والله که لاغست ای کیا',
 'خامش که بس مستعجلم، رفتم سوی پای علم کاغذ بنه بشکن قلم، ساقی درآمد، الصلا',
 '',
 'ای طایران قدس را عشقت فزوده بال ها در حلقه ی سودای تو، روحانیان را حال ها',
 'در "لا احب الافلین"، پاکی ز صورت ها یقین در دیده های غیب بین، هر دم ز تو تمثال ها',


In [10]:
!pip install hazm
from hazm import word_tokenize

     |████████████████████████████████| 316 kB 7.7 MB/s 
     |████████████████████████████████| 1.4 MB 39.4 MB/s 
     |████████████████████████████████| 233 kB 37.8 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394486 sha256=30d6af13ce27b3f43c01b517b146947eb40fad92c0a771f594c1ed2a1b4aef1b
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154471 sha256=725ec8c7af2fb7cc839d0280259584fde0eacfc05c6ce34a47f677ded995a83a
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [11]:
word_tokenized = [word_tokenize(t) for t in text]
word_tokenized_filtered = [[w for w in sentence if w not in stopwords] for sentence in word_tokenized]

In [12]:
#example
word_tokenized_filtered[14]

['کوه',
 'غمت',
 'بشکافته',
 'وآن',
 'غم',
 'دل',
 'درتافته',
 'قطره',
 'خونی',
 'فضلت',
 'افضال']

In [13]:
sentences = [' '.join(sentence) for sentence in word_tokenized_filtered]
sentences = [sentence for sentence in sentences if sentence != '']

In [14]:
len(sentences)

4361

In [15]:
sentences[0]

'رستخیز رحمت منتها آتشی افروخته بیشه اندیشه'

In [16]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

In [17]:
import pickle
with open('tokenizer.h5', 'wb') as f:
    pickle.dump(tokenizer, f)

In [18]:
tokenizer.word_index

{'جان': 1,
 'دل': 2,
 'عشق': 3,
 'گر': 4,
 'ست': 5,
 'آب': 6,
 'چشم': 7,
 'جهان': 8,
 'اندر': 9,
 'شمس': 10,
 'دست': 11,
 'مست': 12,
 'گل': 13,
 'کز': 14,
 'زان': 15,
 'عقل': 16,
 'یار': 17,
 'غم': 18,
 'آتش': 19,
 'شب': 20,
 'جمله': 21,
 'تن': 22,
 'آنک': 23,
 'کان': 24,
 'پر': 25,
 'عالم': 26,
 'مه': 27,
 'نور': 28,
 'ره': 29,
 'خدا': 30,
 'خاک': 31,
 'شاه': 32,
 'باده': 33,
 'کار': 34,
 'تبریز': 35,
 'تویی': 36,
 'روح': 37,
 'عاشق': 38,
 'نی': 39,
 'رخ': 40,
 'همچو': 41,
 'ماه': 42,
 'لطف': 43,
 'زین': 44,
 'خون': 45,
 'زر': 46,
 'پا': 47,
 'ساقی': 48,
 'وز': 49,
 'کف': 50,
 'شکر': 51,
 'مر': 52,
 'خانه': 53,
 'همی': 54,
 'گوش': 55,
 'کیست': 56,
 'ذره': 57,
 'بادا': 58,
 'خورشید': 59,
 'آسمان': 60,
 'بحر': 61,
 'پرده': 62,
 'آرزوست': 63,
 'ای': 64,
 'وان': 65,
 'زمین': 66,
 'شه': 67,
 'کاین': 68,
 'بهر': 69,
 'سخن': 70,
 'عاشقان': 71,
 'تبریزی': 72,
 'باغ': 73,
 'چرخ': 74,
 'جام': 75,
 'گشته': 76,
 'درد': 77,
 'ور': 78,
 'آفتاب': 79,
 'پای': 80,
 'سو': 81,
 'نفس': 82,
 'زند': 83,
 '

In [19]:
encoded = tokenizer.texts_to_sequences(sentences)

In [20]:
encoded

[[2431, 374, 1423, 468, 1424, 1177, 189],
 [217, 512, 1425, 434, 512, 3687, 512, 1178, 375, 30],
 [59, 2432, 36, 653, 2433, 36, 862, 36, 324, 36, 1423, 3688],
 [111, 3689, 189, 1179, 1180, 654],
 [37, 3690, 3691, 288, 183, 405, 5, 1426, 68, 1000, 2434, 238],
 [15, 1426, 739, 1797, 572, 12, 3692, 12, 346, 2435],
 [3693, 655, 16, 218, 3694, 655, 3695, 14, 69, 346, 3696, 1001, 435],
 [863, 123, 1798, 376, 1002, 1798, 3697, 199, 1798, 436, 3698, 157, 3699],
 [3700, 112, 55, 1, 3701, 405, 1427, 1, 406, 3702, 252, 307, 3703, 1799],
 [265, 3704, 740, 80, 288, 2436, 347, 656, 1800, 48, 741, 229],
 [3705, 1428, 469, 3706, 470, 169, 348, 3707],
 [157, 2437, 2438, 1429, 239, 190, 2439],
 [573, 1801, 31, 308, 45, 1802, 3708, 1181, 42, 513],
 [150, 2440, 2441, 2434, 18, 2, 3709, 325, 1003, 3710, 3711],
 [3712, 3713, 3714, 15, 1004, 240, 3715, 9, 2442, 3716],
 [657, 3717, 1182, 3718, 377, 1, 3719, 3720, 76, 658],
 [470, 1430, 2443, 1803],
 [2444, 134, 3721, 134, 3722, 3723, 3724, 46, 3725, 139, 1804

In [21]:
# total number of words in the corpus
num_all_words = sum(len(s) for s in encoded) 
# total number of unique words in the corpus
num_unique_words = len(tokenizer.word_index) + 1  

In [22]:
num_all_words, num_unique_words

(34839, 8644)

In [23]:
# Parameters
window_size = 2 
window_size_corpus = 4

In [24]:
def generate_data(corpus, window_size, num_unique_words):
    maxlen = window_size * 2
    all_inputs = []
    all_outputs = []
    for words in corpus:
      len_words = len(words)
      for index,w in enumerate(words):
        s = index - window_size
        e = index + window_size + 1
        for i in range(s, e):
            if i != index and 0 <= i < len_words:
              all_inputs.append(w)      
              all_outputs.append(to_categorical(words[i], num_unique_words))

    return (np.array(all_inputs), np.array(all_outputs))

In [25]:
# Create training data
X_train, y_train = generate_data(encoded, window_size, num_unique_words)
X_train.shape, y_train.shape

((113192,), (113192, 8644))

In [26]:
X_train, y_train

(array([2431, 2431,  374, ...,   67, 1634, 1634]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32))

In [27]:
dim = 50
#create model
model = Sequential()
model.add(Embedding(num_unique_words, dim, input_length=1, embeddings_initializer='glorot_uniform'))
model.add(Reshape((dim,)))
model.add(Dense(30, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(num_unique_words, activation='softmax', kernel_initializer='glorot_uniform'))

In [28]:
#compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 50)             432200    
                                                                 
 reshape (Reshape)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 30)                1530      
                                                                 
 dense_1 (Dense)             (None, 10)                310       
                                                                 
 dense_2 (Dense)             (None, 8644)              95084     
                                                                 
Total params: 529,124
Trainable params: 529,124
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train, y_train, epochs=50, verbose=1, batch_size=64, validation_split=0.2)

Epoch 1/20
1415/1415 [==============================] - 32s 22ms/step - loss: 8.1475 - accuracy: 0.0197 - val_loss: 8.5343 - val_accuracy: 0.0135
Epoch 2/20
1415/1415 [==============================] - 30s 21ms/step - loss: 7.9041 - accuracy: 0.0203 - val_loss: 8.7424 - val_accuracy: 0.0135
Epoch 3/20
1415/1415 [==============================] - 31s 22ms/step - loss: 7.8712 - accuracy: 0.0203 - val_loss: 8.9273 - val_accuracy: 0.0135
Epoch 4/20
1415/1415 [==============================] - 32s 23ms/step - loss: 7.8437 - accuracy: 0.0203 - val_loss: 9.0629 - val_accuracy: 0.0135
Epoch 5/20
1415/1415 [==============================] - 30s 21ms/step - loss: 7.7975 - accuracy: 0.0202 - val_loss: 9.2174 - val_accuracy: 0.0141
Epoch 6/20
1415/1415 [==============================] - 30s 21ms/step - loss: 7.7297 - accuracy: 0.0213 - val_loss: 9.3342 - val_accuracy: 0.0148
Epoch 7/20
1415/1415 [==============================] - 29s 20ms/step - loss: 7.6511 - accuracy: 0.0224 - val_loss: 9.5009 -

In [ ]:
plt.plot(history.history['loss'], color='green', label='Train Data')
plt.plot(history.history['val_loss'], color='blue', label='Validation Data')

In [ ]:
plt.plot(history.history['accuracy'], color='green', label='Train Data')
plt.plot(history.history['val_accuracy'], color='blue', label='Validation Data')

In [ ]:
weights = model.get_weights()

In [ ]:
weights[0][1]

array([-1.6472973e-01, -7.5903453e-02,  1.3424253e-01, -1.0716214e-01,
       -7.9186432e-02,  1.7673604e-01,  5.4489460e-02, -1.5380678e-01,
       -5.5131916e-02, -7.8720093e-02, -4.7565106e-02, -2.5487730e-02,
        1.1627479e-01,  1.3457619e-01, -7.4619204e-02, -8.3930902e-02,
        1.0412177e-02, -1.0616207e-01, -5.4638233e-02, -1.3879861e-01,
       -1.1484641e-01, -8.2385927e-02,  3.5453971e-02,  1.8596296e-04,
       -2.1902734e-01,  6.9739390e-03,  7.9976574e-02,  2.0844715e-02,
       -3.1725259e-03,  1.0041354e-01, -7.8423552e-02,  1.1067058e-01,
       -1.8597038e-02, -8.0600150e-02, -6.1362281e-02,  1.7614873e-01,
       -1.1996239e-01,  2.2100950e-02, -7.7586941e-02, -1.4010403e-01,
        2.2276722e-01, -8.2245588e-02,  5.3854130e-02, -9.0567917e-02,
        1.8322977e-01,  5.5091973e-02, -8.0183670e-02, -5.8288794e-02,
        6.4455509e-02, -1.6281324e-02], dtype=float32)

In [ ]:
np.argmax(model.predict([750])[0])

1

In [ ]:
def embed(word, embedding, vocab_size=num_unique_words, tokenizer=tokenizer):
    int_word = tokenizer.texts_to_sequences([word])[0]
    # get the one-hot encoding of the word
    bin_word = to_categorical(int_word, num_unique_words)
    return np.dot(bin_word, embedding)

In [ ]:
embeddings = model.get_weights()

In [ ]:
embeded = embed('جنگ', embeddings[0])

In [ ]:
embeded

array([[-1.28624037e-01, -1.16462557e-04, -2.28130907e-01,
         4.00327504e-01,  2.10170239e-01,  2.42990911e-01,
        -9.25571248e-02, -1.48412604e-02,  1.35965705e-01,
         1.02843523e-01,  1.57281712e-01, -1.67983592e-01,
         6.86881095e-02,  2.09082991e-01,  2.67621100e-01,
        -2.06389606e-01, -3.33527297e-01,  1.05879322e-01,
         5.78717649e-01,  4.38724428e-01,  1.04723752e-01,
         1.99940994e-01, -2.48058483e-01,  2.67386585e-01,
        -2.86274254e-01, -1.90090001e-01,  8.94444212e-02,
        -1.17357075e-01,  1.09209687e-01,  2.75773197e-01,
        -2.16351092e-01, -8.33173454e-01,  7.21861124e-02,
        -6.31884933e-02, -3.47010642e-01, -9.16693568e-01,
        -8.17242190e-02, -5.00620231e-02, -1.58360712e-02,
         5.98888278e-01, -1.47603989e-01,  7.48725468e-03,
        -2.63392627e-01,  1.51379272e-01, -4.13282439e-02,
         2.19562843e-01,  4.11928803e-01, -9.47028697e-02,
         4.66208637e-01, -2.19045565e-01]], dtype=float3

In [ ]:
predicted = model.predict(embeded[0])

In [ ]:
predicted

array([[2.5271490e-07, 4.6254030e-05, 5.0225779e-05, ..., 2.5210852e-07,
        2.5104890e-07, 2.5587579e-07],
       [2.5271490e-07, 4.6254030e-05, 5.0225779e-05, ..., 2.5210852e-07,
        2.5104890e-07, 2.5587579e-07],
       [2.5271490e-07, 4.6254030e-05, 5.0225779e-05, ..., 2.5210852e-07,
        2.5104890e-07, 2.5587579e-07],
       ...,
       [2.5271490e-07, 4.6254030e-05, 5.0225779e-05, ..., 2.5210852e-07,
        2.5104890e-07, 2.5587579e-07],
       [2.5271490e-07, 4.6254030e-05, 5.0225779e-05, ..., 2.5210852e-07,
        2.5104890e-07, 2.5587579e-07],
       [2.5271490e-07, 4.6254030e-05, 5.0225779e-05, ..., 2.5210852e-07,
        2.5104890e-07, 2.5587579e-07]], dtype=float32)